# Sentimet Analysis

Demo tutorial for how to use nlp_toolkit to train classification model and predict new samples. The task we choose is sentiment binary classification.

The dataset is crawled from Kanzhun.com and Dajie.com, which is about company pros and cons.

Available models:
    1. Bi-LSTM Attention
    2. Multi Head Self Attention
    3. TextCNN
    4. DPCNN

In [1]:
import sys
sys.path.append('../')
from nlp_toolkit import Dataset, Classifier

/opt/wangyilei/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Data Processing

### Load config dict

In [2]:
import yaml
config = yaml.load(open('../nlp_toolkit/config_classification.yaml'))

### Load Data

In [3]:
dataset = Dataset(fname='../data/company_pro_con.txt', task_type='classification', mode='train', config=config)

2018-11-18 23:23:25,341 - data.py[line:130] - INFO: data loaded


In [4]:
for x, y in zip(dataset.texts[0:10], dataset.labels[0:10]):
    print(x, y)

进去 前 许诺 的 工资 给 的 高 0
校园 环境 优美 ， 美女 很多 ， 适合 居住 ， 食堂 饭菜 便宜 ， 操场 好 ， 可以 天天 运动 0
老板 人 很好 老 员工 会 各种 教 你 东西 ， 而且 不会 有所 保留 薪水 在 大连 还 算 可以 0
人员 比较 多 ， 复杂   办公室 容易 形成 拉帮结派 不利于 企业 发展 1
出差 太多 了 。 在 现场 开发 很苦 逼 。 1
公司 目前 地理位置 不 太 理想 ， 离 城市 中心 较 远点 。 1
公司 的 技术 水平 国内 顶尖 ， 十几 年 的 资历 ， 制作 的 作品 几乎 都 是 精品 ， 参与 过 很多 知名 项目 。 0
工作 流程 复杂     个人 上升 空间 有限     新产品 的 创新 能力 有限   组织 架构 稍 显 臃肿 1
无偿 加班 ， 加班 多 ， 没 加班费 ， 压力 很大 1
环境 比较 轻松 ， 跟 项目 走 ， 能 学 不少 专业 知识 ， 经验 很 重要 0


### Transform data to index

In [5]:
# if we want to use pre_trained embeddings, we need a gensim-format embedding file
x, y, config = dataset.transform()
print(x['token'].shape, y.shape)

2018-11-18 23:23:31,281 - data.py[line:215] - INFO: texts and labels have been transformed to number index
2018-11-18 23:23:36,217 - utilities.py[line:241] - INFO: OOV rate: 0.00 %
2018-11-18 23:23:36,239 - data.py[line:221] - INFO: Loaded Pre_trained Embeddings


(94635, 100) (94635, 2)


if your want to see the vocab and label index mapping dict

In [6]:
# dataset.transformer._word_vocab._token2id

In [7]:
# dataset.transformer._label_vocab._token2id

In [8]:
transformer = dataset.transformer

## Classifier Training

### Define classifier

avialiable models: model_name_list = ['dpcnn', 'text_cnn', 'bi_lstm_att', 'multi_head_self_att']

In [9]:
model_name='multi_head_self_att'
# if you want to get attention weights during prediction, please set return_attention=True, only for bi_lstm_att model
# config['model'][model_name]['return_att'] = True
# text_cnn and dpcnn currently not support bucket sequence type
text_classifier = Classifier(model_name=model_name, transformer=transformer, seq_type='bucket', config=config)

### Train Model

In [10]:
trained_model = text_classifier.train(x, y)

2018-11-18 23:23:36,785 - trainer.py[line:129] - INFO: multi_head_self_att model structure...
2018-11-18 23:23:36,805 - trainer.py[line:139] - INFO: train/valid set: 75708/18927
2018-11-18 23:23:36,806 - trainer.py[line:89] - INFO: use bucket sequence to speed up model training
2018-11-18 23:23:36,809 - sequence.py[line:346] - INFO: Training with 99 non-empty buckets


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
token (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
token_embeddings (Embedding)    (None, None, 300)    7333200     token[0][0]                      
__________________________________________________________________________________________________
position_embedding (Position_Em (None, None, 300)    0           token_embeddings[0][0]           
__________________________________________________________________________________________________
self_attention_0 (Multi_Head_At (None, None, 128)    115200      position_embedding[0][0]         
                                                                 position_embedding[0][0]         
          

2018-11-18 23:23:37,168 - sequence.py[line:346] - INFO: Training with 99 non-empty buckets


mointor training process using f1 score
Successfully made a directory: models/multi_head_self_att_201811182323
using Early Stopping
using Reduce LR On Plateau
tracking loss history and metrics


2018-11-18 23:23:37,584 - trainer.py[line:170] - INFO: saving model parameters and transformer to models/multi_head_self_att_201811182323


model hyperparameters:
 {'nb_classes': 2, 'nb_tokens': 24444, 'maxlen': None, 'nb_head': 8, 'head_size': 16, 'embedding_dim': 300, 'nb_transfomer': 2, 'pos_embed': True, 'final_dropout_rate': 0.5}
Epoch 1/25
1236/1236 [==============================] - 15s 12ms/step - loss: 0.4571 - acc: 0.8911 - val_loss: 0.2741 - val_acc: 0.9245
0 - f1: 92.79
1 - f1: 92.07

Epoch 00001: f1 improved from -inf to 0.92452, saving model to models/multi_head_self_att_201811182323/model_weights_01_0.9245_0.9245.h5
Epoch 2/25
1236/1236 [==============================] - 11s 9ms/step - loss: 0.2448 - acc: 0.9337 - val_loss: 0.2445 - val_acc: 0.9283
0 - f1: 92.88
1 - f1: 92.36

Epoch 00002: f1 improved from 0.92452 to 0.92633, saving model to models/multi_head_self_att_201811182323/model_weights_02_0.9283_0.9263.h5
Epoch 3/25
1236/1236 [==============================] - 12s 10ms/step - loss: 0.1862 - acc: 0.9533 - val_loss: 0.2430 - val_acc: 0.9311
0 - f1: 93.44
1 - f1: 92.85

Epoch 00003: f1 improved from 0.

### 10-fold training

In [11]:
config['train']['train_mode'] = 'fold'
text_classifier_new = Classifier(model_name=model_name, transformer=transformer, seq_type='basic', config=config)
text_classifier_new.train(x, y)

2018-11-18 23:24:52,792 - trainer.py[line:185] - INFO: 10-fold starts!



------------------------ fold 0------------------------


2018-11-18 23:24:53,175 - trainer.py[line:198] - INFO: multi_head_self_att model structure...


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
token (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
token_embeddings (Embedding)    (None, None, 300)    7333200     token[0][0]                      
__________________________________________________________________________________________________
position_embedding (Position_Em (None, None, 300)    0           token_embeddings[0][0]           
__________________________________________________________________________________________________
self_attention_0 (Multi_Head_At (None, None, 128)    115200      position_embedding[0][0]         
                                                                 position_embedding[0][0]         
          

KeyboardInterrupt: 

## Predict New Samples

## Load data and transformer

In [12]:
dataset = Dataset(mode='predict', fname='../data/company_pro_con_predict.txt',
                  task_type='classification',
                  tran_fname='models/multi_head_self_att_201811182323/transformer.h5')
x_seq = dataset.transform()

2018-11-18 23:26:28,469 - data.py[line:96] - INFO: transformer loaded
2018-11-18 23:26:28,673 - data.py[line:130] - INFO: data loaded


data transformer loaded


## Load model

In [14]:
text_classifier = Classifier('multi_head_self_att', dataset.transformer)
text_classifier.load(weight_fname='models/multi_head_self_att_201811182323/model_weights_03_0.9311_0.9316.h5',
                     para_fname='models/multi_head_self_att_201811182323/model_parameters.json')

model loaded


## predict samples

In [15]:
y_pred, attention = text_classifier.predict(x_seq)

KeyError: 'word'

### attention visualization

currently only support for bi_lstm_att model

In [ ]:
x_len = x_seq['length']
attention_true = [attention[i][:x_len[i]] for i in range(len(x_len))]

In [ ]:
from nlp_toolkit import visualization as vs

In [ ]:
vs.mk_html(dataset.texts[2].split(), attention_true[2])

<span style="background-color: #FFFEFE">老板</span> <span style="background-color: #FFFEFE">人</span> <span style="background-color: #FFD6D6">很好</span> <span style="background-color: #FFFCFC">老</span> <span style="background-color: #FFFEFE">员工</span> <span style="background-color: #FFFEFE">会</span> <span style="background-color: #FFFBFB">各种</span> <span style="background-color: #FFD3D3">教</span> <span style="background-color: #FFD7D7">你</span> <span style="background-color: #FFECEC">东西</span> <span style="background-color: #FFFCFC">，</span> <span style="background-color: #FFFEFE">而且</span> <span style="background-color: #FFD4D4">不会</span> <span style="background-color: #FFFEFE">有所</span> <span style="background-color: #FFFEFE">保留</span> <span style="background-color: #FFFEFE">薪水</span> <span style="background-color: #FFFEFE">在</span> <span style="background-color: #FFFEFE">大连</span> <span style="background-color: #FFFEFE">还</span> <span style="background-color: #FFEDED">算</span> <span style="background-color: #FFD5D5">可以</span><br><br>

or you can write all results to html file and open it in a browser

In [ ]:
vs.attention_visualization(dataset.texts, attention_true, x_len, output_fname='result.html')